In [2]:
!pip install aimodelshare --upgrade -q
!pip install dabl
!pip install mljar-supervised

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
mljar-supervised 0.11.3 requires scikit-learn>=1.0, but you have scikit-learn 0.24.2 which is incompatible.
dabl 0.2.4 requires scikit-learn>=1.0, but you have scikit-learn 0.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of t

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import aimodelshare as ai
import dabl
import numpy as np
import pandas as pd

X_train, X_test, y_train, y_test, example_data, y_test_labels = ai.import_quickstart_data("titanic")

types = dabl.detect_types(X_train)
indices = list(types.index)
names = types.columns.to_numpy()
types = types.to_numpy()

d = dict()
for index, tpes in zip(indices, types):
  for name, tpe in zip(names, tpes):
    if tpe:
      d[index] = name
  
X_train_processed = dabl.clean(X_train, type_hints=d)
X_test_processed = dabl.clean(X_test, type_hints=d)

preprocessor = dabl.EasyPreprocessor()
preprocessor.fit(X_train_processed)
X_train_processed = preprocessor.transform(X_train_processed)
preprocessor.fit(X_test_processed)
X_test_processed = preprocessor.transform(X_test_processed)

y_train_labels_processed = np.asarray(list(map(lambda x: 0 if x == y_test_labels[0] else 1, y_train))).reshape(len(X_train), 1)
y_test_labels_processed = np.asarray(list(map(lambda x: 0 if x == y_test_labels[0] else 1, y_test)))

titanic = pd.DataFrame(np.concatenate((X_train_processed, y_train_labels_processed), axis=1), columns=["feature_" + str(i) for i in range(X_train_processed.shape[1])] + ["classification"])
ec = dabl.SimpleClassifier(random_state=0).fit(titanic, target_col="classification")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.
Running DummyClassifier(random_state=0)
accuracy: 0.625 average_precision: 0.375 roc_auc: 0.500 recall_macro: 0.500 f1_macro: 0.384
=== new best DummyClassifier(random_state=0) (using recall_macro):
accuracy: 0.625 average_precision: 0.375 roc_auc: 0.500 recall_macro: 0.500 f1_macro: 0.384

Running GaussianNB()
accuracy: 0.765 average_precision: 0.764 roc_auc: 0.820 recall_macro: 0.764 f1_macro: 0.755
=== new best GaussianNB() (using recall_macro):
accuracy: 0.765 average_precision: 0.764 roc_auc: 0.820 recall_macro: 0.764 f1_macro: 0.755

Running MultinomialNB()
accuracy: 0.770 average_precision: 0.810 roc_auc: 0.834 recall_macro: 0.763 f1_macro: 0.758
Running DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=0)
accuracy: 0.781 average_precision: 0.613 roc_auc: 0.763 recall_macro: 0.763 f1_macro

Seems that dabl does not include xgboost, lightgbm...try another automl lib instead

Default algos: catboost, xgboost, random forest, lightgbm...

Automatically test all possible models by setting mode to 'Compete'

In [3]:
import os
import shutil
automl_dir = "AutoML_classifier"
if os.path.exists(automl_dir):
    shutil.rmtree(automl_dir)

In [4]:
!pip install --upgrade matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import aimodelshare as ai
from supervised.automl import AutoML

X_train, X_test, y_train, y_test, example_data, y_test_labels = ai.import_quickstart_data("titanic")

# automl = AutoML(results_path=automl_dir, mode="Compete")
automl = AutoML(results_path=automl_dir)#, mode="Compete")
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.
AutoML directory: AutoML_classifier
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline logloss 0.670136 trained in 1.36 seconds


2_DecisionTree logloss 0.516392 trained in 20.94 seconds
3_Linear logloss 0.464615 trained in 3.15 seconds
* Step default_algorithms will try to check up to 3 models
4_Default_Xgboost logloss 0.439622 trained in 5.45 seconds
5_Default_NeuralNetwork logloss 0.528917 trained in 1.74 seconds
6_Default_RandomForest logloss 0.442645 trained in 7.3 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.427154 trained in 0.84 seconds
AutoML fit time: 54.8 seconds
AutoML best model: Ensemble


In [6]:
from sklearn.metrics import accuracy_score

print(f"Accuracy: {accuracy_score(y_test, predictions)*100.0:.2f}%" )

Accuracy: 79.77%


Save automl model into onnx model is not supported yet

In [7]:
automl.get_leaderboard()

,name,model_type,metric_type,metric_value,train_time
0,1_Baseline,Baseline,logloss,0.670136,3.30
1,2_DecisionTree,Decision Tree,logloss,0.516392,22.36
2,3_Linear,Linear,logloss,0.464615,4.55
3,4_Default_Xgboost,Xgboost,logloss,0.439622,6.95
4,5_Default_NeuralNetwork,Neural Network,logloss,0.528917,3.12
5,6_Default_RandomForest,Random Forest,logloss,0.442645,8.93
6,Ensemble,Ensemble,logloss,0.427154,0.84


In [8]:
selected_models = automl.ensemble.selected_models
models = []
for model_ in selected_models:
  model = model_["model"]
  models.append([model.get_name(), model])
models

[['2_DecisionTree',
 ['3_Linear', <supervised.model_framework.ModelFramework at 0x7f9c332d6a10>],
 ['4_Default_Xgboost',

Get weights

In [9]:
os.chdir(automl_dir)

In [10]:
import glob

loaded_model = []
for model_name, model in models:
  os.chdir(model_name)
  
  model.load("./", "")
  os.chdir("..") 

  loaded_model.append(model)

In [11]:
import numpy as np

for model in loaded_model:
  predictions = model.predict(X_test)
  predict_cols = predictions.columns
  predict_cols = np.asarray([name.split("_")[-1] for name in predict_cols])
  predictions = predict_cols[predictions.to_numpy().argmax(axis=-1)]
  print(f"Accuracy: {accuracy_score(y_test, predictions)*100.0:.2f}%" )

Accuracy: 77.10%
Accuracy: 75.57%
Accuracy: 78.63%


Transform to onnx with ensemble model

In [12]:
parsed_models = []

for model_detail in models:
  name, model = model_detail
  preprocessor = model.preprocessings[0]
  learner = model.learners[0].model
  parsed_models.append((preprocessor, learner))

In [46]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from skl2onnx import update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes)
from xgboost import XGBClassifier
from onnxmltools.convert.xgboost.operator_converters.XGBoost import (
    convert_xgboost)
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster


preprocessor = parsed_models[0][0]
X_train_transformed, y_train_transformed, _ = preprocessor.fit_and_transform(X_train, y_train)
X_test_transformed, y_test_transformed, _ = preprocessor.fit_and_transform(X_test, y_test)

model_pipelines = []


for preprocessor, model in parsed_models:
  # preprocessor.fit = preprocessor.fit_and_transform

  # # Not a good idea
  # def transforms(X_validation):
  #   try:
  #     return preprocessor.transform(X_validation, y_validation=None)[0]
  #   except:
  #     return preprocessor.transform(X_validation)
  # preprocessor.transform = transforms
  if not hasattr(model, "fit"):
    import xgboost
    if isinstance(model, xgboost.core.Booster):
      new_model = xgboost.XGBClassifier()
      new_model._Booster = model

      update_registered_converter(
                      XGBClassifier, 'XGBoostXGBClassifier',
                      calculate_linear_classifier_output_shapes, convert_xgboost,
                      options={'nocl': [True, False], 'zipmap': [True, False, 'columns']})
    else:
      raise NotImplementedError
    assert hasattr(model, "predict")# and hasattr(preprocessor, "fit") and hasattr(preprocessor, "transform")
    pipeline = new_model
  else:
    pipeline = model
  model_pipelines.append(pipeline)


estimators = [("model_"+str(i), pipeline) for i, pipeline in enumerate(model_pipelines)]

stacked_classifier = StackingClassifier(estimators=estimators, 
            final_estimator=LogisticRegression())

pipeline = make_pipeline(stacked_classifier)

print(pipeline)
print("success")

pipeline.fit(X_train_transformed.to_numpy(), y_train_transformed)
pipeline.score(X_test_transformed.to_numpy(), y_test_transformed)

Pipeline(steps=[('stackingclassifier',
                 StackingClassifier(estimators=[('model_0',
                                                 DecisionTreeClassifier(max_depth=3,
                                                                        random_state=1235)),
                                                ('model_1',
                                                 LogisticRegression(max_iter=500,
                                                                    n_jobs=-1,
                                                                    tol=0.0005)),
                                                ('model_2',
                                                 XGBClassifier(base_score=0.5,
                                                               booster='gbtree',
                                                               callbacks=None,
                                                               colsample_bylevel=1,
                                     

0.8053435114503816

In [45]:
from aimodelshare.aimsonnx import model_to_onnx
from skl2onnx import convert_sklearn

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]  # Insert correct number of features in preprocessed data

onnx_model = convert_sklearn(
    pipeline, 'pipeline_ensemble',
    initial_type,
    target_opset={'': 12, 'ai.onnx.ml': 2})
# onnx_model = model_to_onnx(pipeline, framework='sklearn',
#               initial_types=initial_type,
#               transfer_learning=False, deep_learning=False)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

[('float_input', FloatTensorType(shape=[None, 5]))]
